In [18]:
import dev.example.food.*
import dev.langchain4j.model.*
import dev.langchain4j.model.embedding.*
import dev.langchain4j.data.embedding.*
import dev.langchain4j.store.embedding.*
import dev.langchain4j.model.embedding.onnx.allminilml6v2.*
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf
%use dataframe
%use lets-plot
%use lets-plot-gt
%useLatestDescriptors


In [41]:
val embeddingModel: EmbeddingModel =   AllMiniLmL6V2EmbeddingModel()//BgeSmallZhV15QuantizedEmbeddingModel() 

//https://kindgeek.com/blog/post/experiments-with-langchain4j-or-java-way-to-llm-powered-applications

infix fun String.similarityTo(text: String): Double {
    val inputOne: Embedding = embeddingModel.embed(this).content()
    val inputTwo: Embedding = embeddingModel.embed(text).content()
    
    return CosineSimilarity.between(inputOne, inputTwo)
}

Line_93.jupyter.kts (5:11 - 17) This class shouldn't be used in Kotlin. Use kotlin.String instead.
Line_93.jupyter.kts (5:37 - 43) This class shouldn't be used in Kotlin. Use kotlin.String instead.
Line_93.jupyter.kts (6:46 - 51) None of the following functions can be called with the arguments supplied: 
public open fun embed(p0: TextSegment!): Response<Embedding!>! defined in dev.langchain4j.model.embedding.EmbeddingModel
public open fun embed(p0: String!): Response<Embedding!>! defined in dev.langchain4j.model.embedding.EmbeddingModel
Line_93.jupyter.kts (7:46 - 51) None of the following functions can be called with the arguments supplied: 
public open fun embed(p0: TextSegment!): Response<Embedding!>! defined in dev.langchain4j.model.embedding.EmbeddingModel
public open fun embed(p0: String!): Response<Embedding!>! defined in dev.langchain4j.model.embedding.EmbeddingModel

In [30]:
"Eggs".similarityTo("egg")

0.9120824129867496

In [40]:
val response = embeddingModel.embed("kingfisher")
val embedding:Embedding = response.content()
embedding.dimension()
response.tokenUsage()


TokenUsage { inputTokenCount = 3, outputTokenCount = null, totalTokenCount = 3 }

In [39]:



val url = "jdbc:postgresql://localhost:5431/langchain"
val username = "user"
val password = "password"

val dbConfig = DatabaseConfiguration(url, username, password)

val tableName = "italianfood"

val df = DataFrame.readSqlTable(dbConfig, tableName, 100)



In [38]:
val referenceVector = Arrays.toString(embeddingModel.embed("Dessert with Berries, Cream and Chocolate").content().vector())
val query = """SELECT * from (WITH temp AS (SELECT (2 - (embedding <=> '%s')) / 2 AS score, embedding_id, embedding, text, metadata FROM %s) SELECT * FROM temp WHERE score >= %s ORDER BY score desc LIMIT %s) as result""".format(
    referenceVector, "italianfood", 0.6, 7
)

DataFrame.readSqlQuery(dbConfig, query).select("text", "score").print(valueLimit = 100)



                                                                                                   text    score
 0  Brownies Desserts [Dark chocolate, Eggs, Blanched whole hazelnuts, Butter, Flour, Sugar, Fine salt] 0,819836
 1 Cream tart Desserts [Flour, Almond flour, Butter, Eggs, Sugar, Fresh liquid cream, Mascarpone, Po... 0,818885
 2           Pastry Cream Desserts [Whole milk, Sugar, Egg yolks, Rice starch, Vanilla pod, Lemon zest] 0,797090
 3   Nutella cheesecake Desserts [Digestive biscuits, Butter, Nutella, Cream cheese, Hazelnut granules] 0,793000
 4 Chocolate Profiteroles Desserts [Water, Whole Milk, Butter, Fine Salt, Sugar, Flour, Eggs, Egg Yo... 0,789423
 5                                          Biscuit dough Desserts [Eggs, Honey, Flour, Sugar, Vanilla] 0,784850
 6 Chocolate Salami Desserts [Dark Chocolate, Butter, Dry Biscuits, Sugar, Rum, Unsweetened Cocoa Po... 0,783595



{Index=[[0, 1, 2, 3, 4]], Vector1=[[1, 2, 3, 4, 5]], Vector2=[[5, 4, 3, 2, 1]], Color1=[[red, blue, green, yellow, purple]], Color2=[[red, blue, green, yellow, purple]]}


In [20]:
import kotlin.math.sqrt

// Sample dataset
val data = listOf(
    Pair(1.0, 1.0),
    Pair(2.0, 2.0),
    Pair(3.0, 3.0),
    Pair(4.0, 4.0),
    Pair(5.0, 5.0)
)

// Function to calculate Euclidean distance
fun euclideanDistance(p1: Pair<Double, Double>, p2: Pair<Double, Double>): Double {
    return sqrt((p1.first - p2.first).pow(2) + (p1.second - p2.second).pow(2))
}

// Function to find k-nearest neighbors
fun kNearestNeighbors(data: List<Pair<Double, Double>>, point: Pair<Double, Double>, k: Int): List<Pair<Double, Double>> {
    return data.sortedBy { euclideanDistance(it, point) }.take(k)
}

// Define the point to find neighbors for
val point = Pair(3.5, 3.5)
val k = 3

// Find k-nearest neighbors
val neighbors = kNearestNeighbors(data, point, k)

// Create a DataFrame for the dataset
val df = dataFrameOf("x", "y")(
    data.map { it.first },
    data.map { it.second }
)

// Create a DataFrame for the neighbors
val neighborsDf = dataFrameOf("x", "y")(
    neighbors.map { it.first },
    neighbors.map { it.second }
)

// Plot the dataset and the neighbors
val plot = letsPlot(df.toMap()) +
        geomPoint(color = "blue", size = 3.0) { x = "x"; y = "y" } +
        geomPoint(data = neighborsDf.toMap(), color = "red", size = 5.0) { x = "x"; y = "y" } +
        geomPoint(data = mapOf("x" to listOf(point.first), "y" to listOf(point.second)), color = "green", size = 5.0) { x = "x"; y = "y" }

plot